### Receiver Side
    Receiver has a image & key for decryption

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import binascii

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [2]:
img = cv2.imread('..\\images\\img_output.png')
key = "//END//"

### Extracting LSB from the Image

In [3]:
img.shape

(1200, 1800, 3)

In [4]:
lsb_plane = np.uint8(img & 1)
lsb_plane = np.ravel(lsb_plane)
lsb_plane.shape

(6480000,)

In [5]:
delimiter = key # Delimiter
hex_end = binascii.hexlify(delimiter.encode())
bin_end = bin(int(hex_end, 16))[2:].zfill(8 * ((len(hex_end) + 1) // 2))
bin_end_arr = np.fromiter(bin_end, dtype=int)
bin_end_arr

array([0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1])

In [6]:
idx = np.where(np.convolve(lsb_plane, bin_end_arr[::-1], mode='valid') == np.sum(bin_end_arr))[0][0]
lsb_plane = lsb_plane[0:idx]
lsb_plane.shape

(256,)

In [7]:
bin_str = np.array2string(lsb_plane, separator='')[1:-1].replace(' ', '')
bin_str = bin_str.replace('\n', '')
bin_str.encode()

b'1110101011011000111010101110000011011001001110000011101110010110111001101100010001010011100110110001010001100100010110110100100000110100100001100101111101111100110010000001001010011100000000011010001100100000110101100100000110111110001010111100010011110000'

In [8]:
enc_msg = hex(int(bin_str, 2))[2:]
enc_msg

'ead8eae0d9383b96e6c4539b14645b4834865f7cc8129c01a320d641be2bc4f0'

### Decrypting the Extracted Encrypted Code

In [9]:
key_bytes = key.encode("utf-8").rjust(16)
iv = key_bytes

cipher = AES.new(key_bytes, AES.MODE_CBC, iv)

enc_msg = bytes.fromhex(enc_msg)
msg = unpad(cipher.decrypt(enc_msg), 16).decode("utf-8")
msg

'This is Secret Message'